In [ ]:
!pip install langdetect googletrans==4.0.0-rc1

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
from string import punctuation
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from langdetect import detect
from googletrans import Translator


In [2]:
channel_data = pd.read_csv("C:/Users/Vasu/Desktop/API projects/youtube_api/youtube_merged.csv")

In [3]:
channel_data.head(10)

,video_id,channelTitle,title,tags,description,viewCount,likeCount,commentCount,duration,definition,caption,date,time,day,tag_count,title_length,comments
0,v_lQaB7zgIA,Nakkalites,Short Trip-ஆ சோதிக்காதிங்க Da😒 | Nakkalites #s...,"['nakkalites channel', 'nakkalites new video',...",For Business \nContact us - business@nakkalite...,14269,1472,2.0,0m 53s,hd,False,2024-07-31,12:30:45,Wednesday,25,51,"['Super comedy 😂', 'Super video 😂']"
1,lXjEFmq-qfw,Nakkalites,இதான் Rough & Tough Dealing-ஆ😂 | Nakkalites #s...,"['Nakkalites', 'nakkalites new video', 'amma a...",For Business \nContact us - business@nakkalite...,10948,732,1.0,0m 55s,hd,False,2024-07-31,08:30:24,Wednesday,28,51,['Please pin me bro 🎉🎉🎉🎉']
2,Q8kWxKCJCZo,Nakkalites,என்னடா பொசுக்குன்னு Uncle-னு சொல்லிட்டீங்க🙄😒 |...,"['nakkalites channel', 'nakkalites new video',...",For Business \nContact us - business@nakkalite...,14287,1372,7.0,0m 35s,hd,False,2024-07-30,12:30:17,Tuesday,25,65,"['Bro I am watch group photo i am cry bro', 'B..."
3,R8Viun7HeAM,Nakkalites,அவன விட்ருங்க என்ன மட்டும் கேட்டுட்டே இருங்க |...,"['Nakkalites', 'nakkalites new video', 'amma a...",For Business \nContact us - business@nakkalite...,17704,872,3.0,0m 50s,hd,False,2024-07-30,08:30:14,Tuesday,28,65,"['Pin me🎉🎉🎉🎉', 'Vintage nakkalites laam chance..."
4,oAL7sMhfOZ0,Nakkalites,சின்ன பசங்களாம் புண்படுத்துறாங்களே😒 | Nakkalit...,"['nakkalites channel', 'nakkalites new video',...",For Business \nContact us - business@nakkalite...,14631,1180,3.0,0m 44s,hd,False,2024-07-29,12:30:41,Monday,25,56,"['Pin me', 'Sasi and arun combo v want back', ..."
5,chYndzcBx_c,Nakkalites,மாப்ள நீயும் அத சொல்லாத Da | Nakkalites #shorts,"['Nakkalites', 'nakkalites new video', 'amma a...",For Business \nContact us - business@nakkalite...,15785,1399,3.0,0m 44s,hd,False,2024-07-29,08:30:11,Monday,28,47,"['🤗', 'Nakkalites back tooo form ahhh💕😘💕💕💕❤❤❤❤..."
6,pEOV3N8FzdY,Nakkalites,போறோம் தெறிக்க விடுறோம் | Nakkalites #shorts,"['Nakkalites', 'nakkalites new video', 'amma a...",For Business \nContact us - business@nakkalite...,8355,654,1.0,0m 50s,hd,False,2024-07-28,12:30:29,Sunday,28,44,['First ❤🎉']
7,XbEsBMxSQMc,Nakkalites,அடுத்த சம்பவம் இருக்கு | Nakkalites #shorts,"['nakkalites', 'nakkalites latest', 'nakkalite...",For Business \nContact us - business@nakkalite...,30462,1800,3.0,0m 45s,hd,False,2024-07-28,08:30:09,Sunday,25,43,"['😂😂😂😂', 'Adutha Aapu Udane Nanbanukku tha😂', ..."
8,PJbnDENJUQE,Nakkalites,அடிப்பாவி அப்போ சொன்னதுலாம் பொய்-ஆ | Nakkalit...,"['Nakkalites', 'nakkalites new video', 'amma a...",For Business \nContact us - business@nakkalite...,25318,1491,3.0,0m 36s,hd,False,2024-07-27,12:30:24,Saturday,28,56,"['😂😂😂', 'Hi ashwin, Amit this side I liked wa..."
9,o0yu7ueG_pI,Nakkalites,அவனுங்க அப்டி பண்ண மாட்டாங்களே | Nakkalites #s...,"['nakkalites', 'nakkalites latest', 'nakkalite...",For Business \nContact us - business@nakkalite...,42157,3916,1.0,0m 54s,hd,False,2024-07-27,08:30:13,Saturday,25,51,['Super😂']


In [4]:
channel_data['channelTitle'].unique()

array(['Nakkalites', 'Village Cooking Channel', 'Madan Gowri'],
      dtype=object)

In [5]:
# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Vasu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Vasu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Vasu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Vasu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# Initialize lemmatizer and stopwords
stop_words = set(stopwords.words('english'))
lzr = WordNetLemmatizer()

# Text Preprocessing Function
def text_processing(text):
    # Basic preprocessing
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(f'[{re.escape(punctuation)}]', '', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text, flags=re.I)
    
    # Tokenize, remove stopwords and lemmatize
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lzr.lemmatize(word) for word in tokens]
    
    return ' '.join(tokens)


In [7]:
# Initialize Sentiment Intensity Analyzer
sentiments = SentimentIntensityAnalyzer()

def analyze_channel_sentiment(channel_name, channel_data):
    print(f"Processing sentiment analysis for {channel_name}...")

    # Explode and preprocess comments
    channel_data = channel_data.explode('comments').dropna(subset=['comments'])
    channel_data['comments'] = channel_data['comments'].apply(lambda text: text_processing(text))

    # Sentiment Analysis
    channel_data["Positive"] = [sentiments.polarity_scores(comment)["pos"] for comment in channel_data["comments"]]
    channel_data["Negative"] = [sentiments.polarity_scores(comment)["neg"] for comment in channel_data["comments"]]
    channel_data["Neutral"] = [sentiments.polarity_scores(comment)["neu"] for comment in channel_data["comments"]]
    channel_data['Compound'] = [sentiments.polarity_scores(comment)["compound"] for comment in channel_data["comments"]]

    score = channel_data["Compound"].values
    sentiment = []
    for s in score:
        if s >= 0.05:
            sentiment.append('Positive')
        elif s <= -0.05:
            sentiment.append('Negative')
        else:
            sentiment.append('Neutral')

    channel_data["Sentiment"] = sentiment

    # Dropping unnecessary columns
    channel_data = channel_data.drop(['Positive', 'Negative', 'Neutral', 'Compound'], axis=1)

    # Encoding Sentiment Labels
    le = LabelEncoder()
    channel_data['Sentiment'] = le.fit_transform(channel_data['Sentiment'])

    # Upsampling Minority Classes
    df_neutral = channel_data[channel_data['Sentiment'] == le.transform(['Neutral'])[0]]
    df_negative = channel_data[channel_data['Sentiment'] == le.transform(['Negative'])[0]]
    df_positive = channel_data[channel_data['Sentiment'] == le.transform(['Positive'])[0]]

    df_negative_upsampled = resample(df_negative, replace=True, n_samples=len(df_positive), random_state=42)
    df_neutral_upsampled = resample(df_neutral, replace=True, n_samples=len(df_positive), random_state=42)

    # Concatenate the upsampled dataframes
    final_data = pd.concat([df_negative_upsampled, df_neutral_upsampled, df_positive])

    # Vectorization
    corpus = final_data['comments'].tolist()
    cv = CountVectorizer(max_features=1500)
    X = cv.fit_transform(corpus).toarray()
    y = final_data['Sentiment'].values

    # Model Training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    classifier = GaussianNB()
    classifier.fit(X_train, y_train)

    # Evaluation
    y_pred = classifier.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    nb_score = accuracy_score(y_test, y_pred)

    # Results
    print(f"Sentiment Analysis Results for {channel_name}:")
    print("Confusion Matrix:\n", cm)
    print("Accuracy:", nb_score)
    print("\n")


In [8]:
# Load Dataset
data = pd.read_csv('C:/Users/Vasu/Desktop/API projects/youtube_api/youtube_merged.csv')

# List of channels
channels = data['channelTitle'].unique()

# Perform sentiment analysis for each channel
for channel in channels:
    channel_data = data[data['channelTitle'] == channel]
    analyze_channel_sentiment(channel, channel_data)


Processing sentiment analysis for Nakkalites...
Sentiment Analysis Results for Nakkalites:
Confusion Matrix:
 [[268   6  28]
 [  0 263  15]
 [ 11  34 245]]
Accuracy: 0.8919540229885058


Processing sentiment analysis for Village Cooking Channel...
Sentiment Analysis Results for Village Cooking Channel:
Confusion Matrix:
 [[63  0  0]
 [ 0 72  0]
 [ 0  0 73]]
Accuracy: 1.0


Processing sentiment analysis for Madan Gowri...
Sentiment Analysis Results for Madan Gowri:
Confusion Matrix:
 [[515  22  86]
 [  0 631   0]
 [182  36 389]]
Accuracy: 0.8248253627082214


